# Package
* Builds docker containers for all microservices

In [ ]:
import JupyterFormatter
JupyterFormatter.enable_autoformat();

In [ ]:
import NBInclude: @nbinclude
@nbinclude("../TrainingAlphas/Alpha.ipynb");

In [ ]:
import ProgressMeter: @showprogress, next!

In [ ]:
function copy(file::String, dst::String)
    mkpath(joinpath(dst, dirname(file)))
    cp(file, joinpath(dst, file))
end;

In [ ]:
function fetch_media_lists(basedir::String)
    app = "$basedir/fetch_media_lists"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/fetch_media_lists", app)
    copy("notebooks/API", app)
    copy("environment", app)
end;

In [ ]:
function import_media_lists(basedir::String)
    app = "$basedir/import_media_lists"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/import_media_lists", app)
    copy("notebooks/ImportDatasets/ImportLists/import_lists.jl", app)
    for m in ["anime", "manga"]
        copy("data/processed_data/$m.csv", app)
        for s in ["mal", "anilist", "kitsu", "animeplanet"]
            copy("data/processed_data/$s.$m.csv", app)
        end
    end
    copy("data/processed_data/training.timestamps.csv", app)
end;

In [ ]:
function bagofwords_jl(basedir::String)
    app = "$basedir/bagofwords_jl"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/bagofwords_jl", app)
    copy("notebooks/TrainingAlphas/Alpha.ipynb", app)
    copy("notebooks/TrainingAlphas/Baseline/get_user_biases.jl", app)
    copy("data/processed_data/status.csv", app)
    copy("data/processed_data/item_counts.csv", app)
    for m in ALL_MEDIUMS
        copy("data/alphas/baseline/v1/streaming/$m/rating/params.jld2", app)
    end
end;

In [ ]:
function nondirectional(basedir::String)
    app = "$basedir/nondirectional"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/nondirectional", app)
    copy("notebooks/TrainingAlphas/Alpha.ipynb", app)
    copy("data/processed_data/status.csv", app)
    copy("data/processed_data/item_counts.csv", app)
    for m in ALL_MEDIUMS
        for x in ["adaptations", "dependencies", "recaps", "related"]
            copy("data/alphas/nondirectional/v1/training/$m/watch/$x", app)
        end
    end
end;

In [ ]:
function bagofwords_py(basedir::String)
    for medium in ALL_MEDIUMS
        for metric in ALL_METRICS
            app = "$basedir/bagofwords_py_$(medium)_$(metric)"
            if ispath(app)
                rm(app; recursive = true)
            end
            cp("notebooks/Microservices/bagofwords_py", app)
            copy("notebooks/TrainingAlphas/BagOfWords/bagofwords.py", app)
            datadir = "data/alphas/bagofwords/v1/streaming/$medium/$metric"
            copy("$datadir/model.pt", app)
            copy("$datadir/config.json", app)
        end
    end
end;

In [ ]:
# function transformer_jl(basedir::String)
#     app = "$basedir/transformer_jl"
#     if ispath(app)
#         rm(app; recursive = true)
#     end
#     cp("notebooks/Microservices/transformer_jl", app)
#     copy("notebooks/TrainingAlphas/AlphaBase.ipynb", "$app/App/src")
#     copy("notebooks/TrainingAlphas/Transformer/Data.ipynb", "$app/App/src")
#     copy("data/alphas/all/Transformer/v1/0/config.json", "$app/App/src")
#     copy("data/processed_data", "$app/App/src")
#     build(app, basedir)
# end;

In [ ]:
# function transformer_py(basedir::String)
#     for medium in ALL_MEDIUMS
#         app = "$basedir/transformer_py_$(medium)"
#         if ispath(app)
#             rm(app; recursive = true)
#         end
#         cp("notebooks/Microservices/transformer_py", app)
#         copy("notebooks/TrainingAlphas/Transformer/transformer.py", app)
#         copy("data/alphas/$(medium)/Transformer/v1/model.pt", app)
#         copy("data/alphas/all/Transformer/v1/0/config.json", app)
#         build(app, basedir)
#     end
# end;

In [ ]:
# function get_endpoints()
#     apps = [
#         [
#             "index",
#             "fetch_media_lists",
#             "compress_media_lists",
#             "nondirectional",
#             "bagofwords_jl",
#             "transformer_jl",
#         ]
#         ["bagofwords_py_$(x)_$(y)" for x in ALL_MEDIUMS for y in ALL_METRICS]
#         ["transformer_py_$(x)" for x in ALL_MEDIUMS]
#     ]
#     settings = get_settings()
#     if get(settings, "cloud_hosting", false)
#         domain = settings["cloud_hosting_domain"]
#         tag = settings["cloud_hosting_tag"]
#         d = Dict()
#         for app in apps
#             name = replace(app, "_" => "-")
#             d[app] = "https://$(name)-$(tag)-$(domain)"
#         end
#         return d
#     else
#         return Dict(app => "http://$(app):8080" for app in apps)
#     end
# end

# function index(basedir::String)
#     app = "$basedir/index"
#     if ispath(app)
#         rm(app; recursive = true)
#     end
#     cp("notebooks/Microservices/index", app)
#     open("environment/endpoints.json", "w") do f
#         write(f, JSON.json(get_endpoints()))
#     end
#     copy("environment/endpoints.json", "$app/App/src")
#     copy("notebooks/TrainingAlphas/AlphaBase.ipynb", "$app/App/src")
#     copy("notebooks/TrainingAlphas/Ensemble/EnsembleInputs.ipynb", "$app/App/src")
#     copy("data/processed_data", "$app/App/src")
#     for m in ALL_MEDIUMS
#         copy("environment/alphas/$m/Linear", "$app/App/src")
#     end
#     build(app, basedir)
# end;

In [ ]:
function package(basedir::String)
    cp("notebooks/Microservices/app", basedir)
    apps = [
        fetch_media_lists,
        import_media_lists,
        bagofwords_jl,
        bagofwords_py,
        nondirectional,
        #
        # transformer_py,
        # transformer_jl,
        # index,
    ]
    for app in apps
        app(basedir)
    end
end;

In [ ]:
cd(joinpath(@__DIR__, "../.."))
basedir = "data/images";

In [ ]:
if ispath(basedir)
    rm(basedir; recursive = true)
end

In [ ]:
package(basedir);

In [ ]:
function qrun(cmd)
    run(pipeline(cmd, stdout = devnull))
end;

# function build(app::String, basedir::String)
#     name = split(app, "/")[end]
#     qrun(`docker build -q -t rsys/$name $basedir/$name`)
# end;

In [ ]:
qrun(`docker system prune -f`);

In [ ]:
# qrun(`docker system prune -af`);